# Model

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 输入是1个通道，输出是6个通道，卷积核为3*3，步进为1， 1
        self.conv1 = nn.Conv2d(1, 6,  (3, 3))
        self.conv2 = nn.Conv2d(6, 16, (3, 3))
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x     
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        features = 1
        for s in size:
            features *= s
        return features
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [48]:
# batch_size, channel, length, width
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0612, -0.0200, -0.0481,  0.0406, -0.0305,  0.0017,  0.0184,  0.0640,
          0.1647, -0.1858]], grad_fn=<AddmmBackward>)


In [35]:
# net.zero_grad()
# out.backward(torch.randn(1, 10))

# Loss Function

In [43]:
out.size()

torch.Size([1, 10])

In [44]:
target = torch.randn(10).view((1, 10))

In [45]:
criterion = nn.MSELoss()
loss = criterion(out, target)

In [46]:
loss

tensor(0.8850, grad_fn=<MseLossBackward>)

# Backprop

In [40]:
net.zero_grad()
print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)
loss.backward()
print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad before backward
tensor([-0.0018, -0.0017,  0.0049,  0.0061, -0.0066,  0.0089])


# Update the weights-1

In [30]:
lr = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * lr)

# Update the weights-2

In [49]:
import torch.optim as optim
# create a optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop
optimizer.zero_grad()  # zero the gradient buffer
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()